In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

In [2]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/RestaurantLessThan20_data_encoded.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 70.78% Yes
After SMOTE upsampling
	Num training samples: 3,147 (49.98% yes)
	Num testing samples:  787 (50.06% yes)


In [3]:
## Replace the unknown values (previously encoded as 0.0) using the most frequent value seen in the TRAINING data
X_train, X_test, train_modes = proj_utils.replace_unknown_with_mode(X_train, X_test, 
                                                                    outfile_prefix='RestaurantLessThan20')

## Testing RandomForestClassifier with Grid Search Cross-validation


In [4]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_samples_split': [2, 3, 4, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_features=2, min_samples_split=4, n_estimators=200,
                       oob_score=True, random_state=42)

In [5]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.8525579917381634
ROC AUC: TEST 0.944040



In [6]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
8,1,2,4,200,0.92314868,0.01115428,0.99958776,0.00009351
7,2,2,4,150,0.92280399,0.01155379,0.99955116,0.00010272
4,3,2,3,150,0.92276358,0.01086488,0.99995822,0.00002599
5,4,2,3,200,0.92257351,0.01109275,0.99996100,0.00002555
25,5,4,2,150,0.92221236,0.00996090,0.99999937,0.00000040
1,6,2,2,150,0.92211562,0.00823018,0.99999950,0.00000038
26,7,4,2,200,0.92197154,0.00967219,0.99999950,0.00000038
2,8,2,2,200,0.92188746,0.00906497,0.99999950,0.00000038
14,9,3,2,200,0.92172132,0.00952730,0.99999950,0.00000038
17,10,3,3,200,0.92165090,0.01006773,0.99996920,0.00001928


In [7]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
7,time,0.06627968
12,minsToCouponDest,0.06263863
10,income,0.05975317
8,age,0.05666291
14,CoffeeHouse,0.05262731
16,RestaurantLessThan20,0.05018302
15,CarryAway,0.04967695
9,education,0.04910678
17,Restaurant20To50,0.04717214
13,Bar,0.04652464


In [8]:
import pickle

with open('model/phase1/RestaurantLessThan20_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open('model/phase1/RestaurantLessThan20_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)